In [1]:
import pandas as pd
from sklearn import preprocessing
from random import random
import math

In [2]:
df=pd.read_csv('spotify.csv')
df.head()

,artist,song,duration,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Britney Spears,Oops!...I Did It Again,211160,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053
1,blink-182,All The Small Things,167066,1999,79,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726
2,Faith Hill,Breathe,250546,1999,66,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859
3,Bon Jovi,It's My Life,224493,2000,78,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992
4,*NSYNC,Bye Bye Bye,200560,2000,65,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.001040,0.0845,0.879,172.656


In [3]:
dfNorms = df.copy()

for column in df.columns:
    try:
        dfNorms[column] = (dfNorms[column] - dfNorms[column].min())/(dfNorms[column].max() - dfNorms[column].min())
    except:
        pass

dfNorms.head()

,artist,song,duration,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Britney Spears,Oops!...I Did It Again,0.264478,0.090909,0.865169,0.735225,0.825230,0.090909,0.744639,0.0,0.037084,0.307363,0.000018,0.401082,0.915499,0.232272
1,blink-182,All The Small Things,0.145673,0.045455,0.887640,0.360520,0.891961,0.000000,0.770630,1.0,0.046310,0.010534,0.000000,0.710162,0.690876,0.588118
2,Faith Hill,Breathe,0.370598,0.045455,0.741573,0.472813,0.467217,0.636364,0.568584,1.0,0.010492,0.177238,0.000000,0.276007,0.256605,0.509441
3,Bon Jovi,It's My Life,0.300402,0.090909,0.876404,0.498818,0.908908,0.000000,0.812877,0.0,0.042330,0.026928,0.000014,0.391461,0.541127,0.397615
4,*NSYNC,Bye Bye Bye,0.235918,0.090909,0.730337,0.573286,0.924796,0.727273,0.776164,0.0,0.051375,0.041784,0.001056,0.075767,0.899454,0.746771


In [4]:
length = len(dfNorms.index)
splitHere = int(length * 0.7)
train = dfNorms[0:splitHere]
test = dfNorms[splitHere:length]

In [5]:
def trainMRModel(trainDF, iterations):
  modelA = [0.5, 0.5, 0.5, 0.5]

  distanceA = calculateDistance(trainDF, modelA)
  print("First model - weights: ", modelA, " distance: ", distanceA)
  
  modelB = modelA[:] 
  for i in range(iterations):
    modelB = take_best_step(trainDF, modelB)
  
  distanceB = calculateDistance(trainDF, modelB)
  print("Final model - weights: ", modelB, " distance: ", distanceB)
  
def take_random_step(model):
  return [
      model[0] + (random() - 0.5), 
      model[1] + (random() - 0.5),
      model[2] + (random() - 0.5),
      model[3] + (random() - 0.5)
  ]

def take_best_step(trainDF, model):
  old_distance = calculateDistance(trainDF, model)
  for i in range(50): 
    new_model = take_random_step(model)
    new_distance = calculateDistance(trainDF, new_model)
    if new_distance < old_distance:
      return new_model
  return model 

def calculateDistance(trainDF, model):
  sum = 0
  for i in range(len(trainDF.index[:10])):
    energy = trainDF.iat[i, 6]
    loudness = trainDF.iat[i, 8]
    valenc = trainDF.iat[i, 14]
    a = model[0]
    b = model[1]
    c = model[2]
    d = model[3]
    numerator = abs((a * energy) + (b * loudness) + (c * valenc) + d)
    denominator = math.sqrt((a * a) + (b * b) + (c * c ) + (d * d))
    distance = numerator / denominator
    sum = sum + distance
 
  return sum

In [7]:
trainMRModel(train, 1500)

First model - weights:  [0.5, 0.5, 0.5, 0.5]  distance:  16.298195182594867
Final model - weights:  [0.3227630507185898, 2.558758206447915, -0.15292762556585027, -2.1479969206864125]  distance:  0.4076984854095624


In [32]:
test.reset_index(drop=True,inplace=True)

dfDe = dfNorms.copy()

for column in dfDe:
    try:
        dfDe[column] = df[column]* (dfDe[column].max() - dfDe[column].min()) + dfDe[column].min()
    except:
        pass

mse=0
for i in range(len(dfDe)):
    energy = dfDe.iat[i, 6]
    loudness= dfDe.iat [i, 8]
    valenc= dfDe.iat [i, 14]
    pop= dfDe.iat [i, 4]
    mse +=((0.3227630507185898 * energy + 2.558758206447915 + -0.15292762556585027 * loudness + -2.1479969206864125 * valenc)-pop)**2
mse = mse/len(dfDe)
print(mse)

3752.844277863086


In [23]:
dfDe.reset_index(drop=True,inplace=True)
dfDe.head()

,artist,song,duration,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Britney Spears,Oops!...I Did It Again,211160.0,2000.0,77.0,0.751,0.834,1.0,-5.444,0.0,0.0437,0.3000,0.000018,0.3550,0.894,95.053
1,blink-182,All The Small Things,167066.0,1999.0,79.0,0.434,0.897,0.0,-4.918,1.0,0.0488,0.0103,0.000000,0.6120,0.684,148.726
2,Faith Hill,Breathe,250546.0,1999.0,66.0,0.529,0.496,7.0,-9.007,1.0,0.0290,0.1730,0.000000,0.2510,0.278,136.859
3,Bon Jovi,It's My Life,224493.0,2000.0,78.0,0.551,0.913,0.0,-4.063,0.0,0.0466,0.0263,0.000013,0.3470,0.544,119.992
4,*NSYNC,Bye Bye Bye,200560.0,2000.0,65.0,0.614,0.928,8.0,-4.806,0.0,0.0516,0.0408,0.001040,0.0845,0.879,172.656


In [29]:
sums=abs(energy * 0.54 + -5.692 * loudness + valenc * 121.932 + 2.1479969206864125)
print(sums)

89.89279692068642
